# 🌊 ODATIS/AVISO FSLE Data Downloader

This notebook downloads **Finite-Size Lyapunov Exponent (FSLE)** data from the AVISO/ODATIS THREDDS server.

**Instructions:** Run cells 1, 2, and 3 in order. Cell 3 displays the interactive interface.

In [9]:
# =============================================================================
# CELL 1: Install required packages
# =============================================================================
import sys
import subprocess

def install_packages():
    """Install required packages in the current kernel environment."""
    packages = [
        "netCDF4",
        "xarray",
        "ipywidgets",
        "pandas",
        "requests"
    ]
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "uv", "--quiet"])
        cmd = [sys.executable, "-m", "uv", "pip", "install", "--python", sys.executable] + packages + ["--quiet"]
        subprocess.check_call(cmd)
    except Exception:
        cmd = [sys.executable, "-m", "pip", "install"] + packages + ["--quiet"]
        subprocess.check_call(cmd)
    
    print("✅ Libraries installed successfully!")

install_packages()

✅ Libraries installed successfully!


In [10]:
# =============================================================================
# CELL 2: Imports and configuration
# =============================================================================
import os
import tempfile
import requests
from requests.auth import HTTPBasicAuth
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from pathlib import Path

# --- ODATIS/AVISO CREDENTIALS ---
AVISO_USER = 'tds@odatis-ocean.fr'
AVISO_PASS = 'odatis'
BASE_NCSS_URL = "https://tds-odatis.aviso.altimetry.fr/thredds/ncss/grid/dataset-duacs-dt-global-allsat-madt-fsle"

print("🔗 Configuration loaded.")
print(f"📡 Target: {BASE_NCSS_URL}")

🔗 Configuration loaded.
📡 Target: https://tds-odatis.aviso.altimetry.fr/thredds/ncss/grid/dataset-duacs-dt-global-allsat-madt-fsle


In [11]:
# =============================================================================
# CELL 3: Complete download interface (all-in-one)
# =============================================================================

import os
import requests
import pandas as pd
import xarray as xr
import ipywidgets as widgets
from pathlib import Path
from datetime import datetime
from requests.auth import HTTPBasicAuth
from IPython.display import display, clear_output

# -----------------------------------------------------------------------------
# DOWNLOAD ENGINE
# -----------------------------------------------------------------------------
def parse_date_file(filepath):
    """
    Parse a text file containing dates, one per line.
    """
    dates = []
    skip_prefixes = ('#', '=', '-')
    skip_words = ('lista', 'total', 'date', 'datas', 'unique', 'unica')
    
    with open(filepath, 'r') as f:
        for line in f:
            # Remove inline comments (everything after #)
            if '#' in line:
                line = line.split('#')[0]
            
            line = line.strip()
            
            # Skip empty lines
            if not line:
                continue
            
            # Skip lines starting with special characters
            if line.startswith(skip_prefixes):
                continue
            
            # Skip header lines (case-insensitive)
            if any(line.lower().startswith(word) for word in skip_words):
                continue
            
            # Try to parse as date
            try:
                dates.append(pd.to_datetime(line))
            except:
                pass  # Skip unparseable lines silently
    
    return dates


def download_fsle_data(lat_range, lon_range, date_mode, date_input, output_dir, 
                       save_mode='individual', progress_widget=None):
    """
    Download FSLE data using direct NCSS URL construction with HTTP Basic Auth.
    Checks if file exists before downloading.
    """
    
    # 1. Parse and validate dates
    dates_to_download = []
    try:
        if date_mode == 'Single Date':
            dates_to_download = [pd.to_datetime(date_input)]
        elif date_mode == 'Date Range':
            start_date, end_date = date_input
            dates_to_download = pd.date_range(start=start_date, end=end_date, freq='D').tolist()
        elif date_mode == 'Date List File':
            if not os.path.exists(date_input):
                return False, f"❌ File not found: {date_input}"
            dates_to_download = parse_date_file(date_input)
    except Exception as e:
        return False, f"❌ Date processing error: {str(e)}"

    if not dates_to_download:
        return False, "❌ No valid dates found to download."
    
    # Sort dates chronologically
    dates_to_download = sorted(dates_to_download)
    print(f"📅 Found {len(dates_to_download)} valid dates in selection")

    # 2. Ensure output directory exists
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    print(f"📂 Output directory: {output_path.absolute()}")
    
    # 3. Check for existing files (LOGIC ADDED HERE)
    dates_to_process = []
    skipped_count = 0
    
    print("-" * 60)
    print("🔍 Checking existing files...")
    
    for dt in dates_to_download:
        # Define expected filename pattern
        expected_file = output_path / f"fsle_{dt.strftime('%Y%m%d')}.nc"
        
        if expected_file.exists():
            # WARN USER AND SKIP
            print(f"🚫 Skipping: {expected_file.name} (File already exists)")
            print(f"   (ℹ️ To download this date again, please delete the file from the folder)")
            skipped_count += 1
        else:
            dates_to_process.append(dt)
            
    print("-" * 60)
    
    if skipped_count > 0:
        print(f"⏭️ Total files skipped: {skipped_count}")
    
    if not dates_to_process:
        return True, f"✅ All {len(dates_to_download)} files already exist. No new downloads needed."
    
    print(f"\n🔄 Starting download for {len(dates_to_process)} NEW file(s)...")
    
    # 4. Setup authenticated session
    # ENSURE AVISO_USER AND AVISO_PASS ARE DEFINED IN PREVIOUS CELLS
    if 'AVISO_USER' not in globals() or 'AVISO_PASS' not in globals():
        # Fallback to hardcoded if not found (or raise error)
        # Using the credentials identified in previous steps
        user_auth = 'tds@odatis-ocean.fr'
        pass_auth = 'odatis'
    else:
        user_auth = AVISO_USER
        pass_auth = AVISO_PASS

    session = requests.Session()
    session.auth = HTTPBasicAuth(user_auth, pass_auth)
    
    if progress_widget:
        progress_widget.max = len(dates_to_process)
        progress_widget.value = 0
    
    downloaded_files = []
    failed_dates = []
    
    # Base URL for NCSS
    BASE_NCSS_URL = "https://tds-odatis.aviso.altimetry.fr/thredds/ncss/grid/dataset-duacs-dt-global-allsat-madt-fsle"

    # 5. Download loop
    for i, dt in enumerate(dates_to_process):
        try:
            t_start = dt.strftime('%Y-%m-%dT00:00:00Z')
            t_end = dt.strftime('%Y-%m-%dT23:59:59Z')
            
            params = {
                'var': ['fsle_max', 'theta_max'],
                'north': lat_range[1],
                'south': lat_range[0],
                'west': lon_range[0],
                'east': lon_range[1],
                'horizStride': 1,
                'time_start': t_start,
                'time_end': t_end,
                'accept': 'netcdf3',
                'addLatLon': 'true'
            }
            
            response = session.get(BASE_NCSS_URL, params=params, timeout=120)
            
            if response.status_code == 200:
                # Save directly to final location
                out_file = output_path / f"fsle_{dt.strftime('%Y%m%d')}.nc"
                with open(out_file, 'wb') as f:
                    f.write(response.content)
                
                downloaded_files.append(out_file)
                print(f"  ✓ Downloaded: {out_file.name}")
            else:
                print(f"  ⚠️ Failed {dt.strftime('%Y-%m-%d')}: HTTP {response.status_code} - {response.reason}")
                failed_dates.append(dt)
                if response.text:
                    print(f"     Server message: {response.text[:150]}...")
                    
        except Exception as e:
            print(f"  ❌ Error {dt.strftime('%Y-%m-%d')}: {str(e)}")
            failed_dates.append(dt)
        
        if progress_widget:
            progress_widget.value = i + 1

    # 6. Build result message
    total_files = len(downloaded_files) + skipped_count
    
    if not downloaded_files and skipped_count == 0:
        return False, "❌ No data was successfully downloaded."
    
    msg = f"\n✅ Process complete!\n"
    msg += f"📁 Output folder: {output_path}\n"
    msg += f"📥 Downloaded: {len(downloaded_files)}\n"
    msg += f"⏭️ Skipped (Existing): {skipped_count}\n"
    
    if failed_dates:
        msg += f"\n⚠️ Failed dates ({len(failed_dates)}): {[d.strftime('%Y-%m-%d') for d in failed_dates[:5]]}"
        if len(failed_dates) > 5:
            msg += f" ... and {len(failed_dates)-5} more"
    
    # 7. Optionally concatenate all files
    if save_mode == 'concatenated' and total_files > 1:
        print("\n💾 Concatenating all files into single NetCDF...")
        try:
            # We look for ALL files in the folder matching pattern, not just downloaded ones
            all_files = sorted(output_path.glob('fsle_*.nc'))
            if all_files:
                ds = xr.open_mfdataset(all_files, combine='by_coords')
                
                d_start = dates_to_download[0].strftime('%Y%m%d')
                d_end = dates_to_download[-1].strftime('%Y%m%d')
                concat_file = output_path / f"fsle_combined_{d_start}_to_{d_end}.nc"
                
                ds.to_netcdf(concat_file)
                msg += f"\n📦 Combined file created: {concat_file.name}"
        except Exception as e:
            msg += f"\n⚠️ Could not concatenate: {str(e)}"
    
    return True, msg


# -----------------------------------------------------------------------------
# FOLDER BROWSER CLASS
# -----------------------------------------------------------------------------
class FolderSelector:
    """Interactive folder selector with navigation and creation capabilities."""
    
    def __init__(self, start_path='.', title='📁 Output Folder'):
        self.current_path = Path(start_path).resolve()
        self.selected_path = self.current_path
        self.title = title
        
        self.path_display = widgets.HTML(value=self._format_path_html())
        
        self.folder_dropdown = widgets.Select(
            options=self._get_folder_options(),
            description='',
            layout=widgets.Layout(width='100%', height='120px')
        )
        
        self.up_btn = widgets.Button(description='⬆️ Up', button_style='info', layout=widgets.Layout(width='80px'))
        self.enter_btn = widgets.Button(description='📂 Enter', button_style='primary', layout=widgets.Layout(width='90px'))
        self.select_btn = widgets.Button(description='✅ Select', button_style='success', layout=widgets.Layout(width='90px'))
        
        self.new_folder_name = widgets.Text(placeholder='New folder name...', layout=widgets.Layout(width='180px'))
        self.create_btn = widgets.Button(description='➕ Create', button_style='warning', layout=widgets.Layout(width='90px'))
        
        self.selected_display = widgets.HTML(value=f"<b>Selected:</b> <code>{self.selected_path}</code>")
        
        self.up_btn.on_click(self._go_up)
        self.enter_btn.on_click(self._enter_folder)
        self.select_btn.on_click(self._select_current)
        self.create_btn.on_click(self._create_folder)
        
        self.widget = widgets.VBox([
            widgets.HTML(f"<h4>{self.title}</h4>"),
            self.path_display,
            self.folder_dropdown,
            widgets.HBox([self.up_btn, self.enter_btn, self.select_btn]),
            widgets.HBox([self.new_folder_name, self.create_btn]),
            self.selected_display
        ])
    
    def _format_path_html(self):
        return f"<b>Current:</b> <code>{self.current_path}</code>"
    
    def _get_folder_options(self):
        try:
            folders = ['[ . ] (current folder)']
            for item in sorted(self.current_path.iterdir()):
                if item.is_dir() and not item.name.startswith('.'):
                    folders.append(f"📁 {item.name}")
            return folders
        except PermissionError:
            return ['[ . ] (current folder)']
    
    def _refresh(self):
        self.path_display.value = self._format_path_html()
        self.folder_dropdown.options = self._get_folder_options()
        self.folder_dropdown.value = self.folder_dropdown.options[0]
    
    def _go_up(self, b):
        parent = self.current_path.parent
        if parent != self.current_path:
            self.current_path = parent
            self._refresh()
    
    def _enter_folder(self, b):
        selection = self.folder_dropdown.value
        if selection and not selection.startswith('[ . ]'):
            folder_name = selection.replace('📁 ', '')
            new_path = self.current_path / folder_name
            if new_path.is_dir():
                self.current_path = new_path
                self._refresh()
    
    def _select_current(self, b):
        self.selected_path = self.current_path
        self.selected_display.value = f"<b>✅ Selected:</b> <code>{self.selected_path}</code>"
    
    def _create_folder(self, b):
        name = self.new_folder_name.value.strip()
        if name:
            new_path = self.current_path / name
            try:
                new_path.mkdir(parents=True, exist_ok=True)
                self.new_folder_name.value = ''
                self.current_path = new_path
                self.selected_path = new_path
                self._refresh()
                self.selected_display.value = f"<b>✅ Created & Selected:</b> <code>{self.selected_path}</code>"
            except Exception as e:
                self.selected_display.value = f"<b>❌ Error:</b> {str(e)}"
    
    def get_selected_path(self):
        return str(self.selected_path)


# -----------------------------------------------------------------------------
# FILE BROWSER CLASS
# -----------------------------------------------------------------------------
class FileSelector:
    """Interactive file selector with navigation capabilities."""
    
    def __init__(self, start_path='.', title='📄 Select File', file_filter=None):
        self.current_path = Path(start_path).resolve()
        self.selected_file = None
        self.title = title
        self.file_filter = file_filter
        
        self.path_display = widgets.HTML(value=self._format_path_html())
        
        self.file_dropdown = widgets.Select(
            options=self._get_items(),
            description='',
            layout=widgets.Layout(width='100%', height='150px')
        )
        
        self.up_btn = widgets.Button(description='⬆️ Up', button_style='info', layout=widgets.Layout(width='80px'))
        self.enter_btn = widgets.Button(description='📂 Enter', button_style='primary', layout=widgets.Layout(width='90px'))
        self.select_btn = widgets.Button(description='✅ Select File', button_style='success', layout=widgets.Layout(width='120px'))
        
        self.selected_display = widgets.HTML(value="<b>Selected:</b> <i>No file selected</i>")
        
        self.up_btn.on_click(self._go_up)
        self.enter_btn.on_click(self._enter_folder)
        self.select_btn.on_click(self._select_file)
        
        self.widget = widgets.VBox([
            widgets.HTML(f"<h4>{self.title}</h4>"),
            self.path_display,
            self.file_dropdown,
            widgets.HBox([self.up_btn, self.enter_btn, self.select_btn]),
            self.selected_display
        ])
    
    def _format_path_html(self):
        return f"<b>Current:</b> <code>{self.current_path}</code>"
    
    def _get_items(self):
        """Get folders and files in current directory."""
        try:
            items = []
            folders = []
            files = []
            
            for item in sorted(self.current_path.iterdir()):
                if item.name.startswith('.'):
                    continue
                if item.is_dir():
                    folders.append(f"📁 {item.name}")
                elif item.is_file():
                    # Apply file filter if specified
                    if self.file_filter:
                        if item.suffix.lower() in self.file_filter:
                            files.append(f"📄 {item.name}")
                    else:
                        files.append(f"📄 {item.name}")
            
            return folders + files if (folders or files) else ['(empty folder)']
        except PermissionError:
            return ['(permission denied)']
    
    def _refresh(self):
        self.path_display.value = self._format_path_html()
        self.file_dropdown.options = self._get_items()
        if self.file_dropdown.options:
            self.file_dropdown.value = self.file_dropdown.options[0]
    
    def _go_up(self, b):
        parent = self.current_path.parent
        if parent != self.current_path:
            self.current_path = parent
            self._refresh()
    
    def _enter_folder(self, b):
        selection = self.file_dropdown.value
        if selection and selection.startswith('📁'):
            folder_name = selection.replace('📁 ', '')
            new_path = self.current_path / folder_name
            if new_path.is_dir():
                self.current_path = new_path
                self._refresh()
    
    def _select_file(self, b):
        selection = self.file_dropdown.value
        if selection and selection.startswith('📄'):
            file_name = selection.replace('📄 ', '')
            self.selected_file = self.current_path / file_name
            self.selected_display.value = f"<b>✅ Selected:</b> <code>{self.selected_file}</code>"
        else:
            self.selected_display.value = "<b>⚠️</b> Please select a file (📄), not a folder"
    
    def get_selected_file(self):
        """Return the selected file path as string, or None if no file selected."""
        return str(self.selected_file) if self.selected_file else None


# -----------------------------------------------------------------------------
# USER INTERFACE
# -----------------------------------------------------------------------------
style = {'description_width': 'initial'}

# Create folder selector for output
folder_selector = FolderSelector(start_path='.', title='📁 Output Folder')

# Create file selector for date list
file_selector = FileSelector(start_path='.', title='📄 Date List File', file_filter=['.txt', '.csv', '.dat'])

# Geographic selection
w_lat = widgets.FloatRangeSlider(
    value=[33, 45], min=-90, max=90, step=0.5,
    description='Latitude (S → N):', style=style,
    layout=widgets.Layout(width='60%'), continuous_update=False
)

w_lon = widgets.FloatRangeSlider(
    value=[-77, -64], min=-180, max=180, step=0.5,
    description='Longitude (W → E):', style=style,
    layout=widgets.Layout(width='60%'), continuous_update=False
)

# Date selection
w_mode = widgets.Dropdown(
    options=['Single Date', 'Date Range', 'Date List File'],
    value='Single Date', description='Date Mode:', style=style
)

w_date_single = widgets.DatePicker(description='Select Date:', value=datetime(2024, 3, 30), style=style)
w_date_start = widgets.DatePicker(description='Start Date:', value=datetime(2024, 3, 30), style=style)
w_date_end = widgets.DatePicker(description='End Date:', value=datetime(2024, 4, 1), style=style)
w_date_range_box = widgets.HBox([w_date_start, w_date_end])

# Save mode selector
w_save_mode = widgets.RadioButtons(
    options=[
        ('Individual files (one per day, resumable)', 'individual'),
        ('Single concatenated file', 'concatenated')
    ],
    value='individual',
    description='Save mode:',
    style=style
)

# Container for date inputs (will switch between picker and file browser)
w_date_container = widgets.VBox([w_date_single])

def on_mode_change(change):
    if change['new'] == 'Single Date':
        w_date_container.children = [w_date_single]
    elif change['new'] == 'Date Range':
        w_date_container.children = [w_date_range_box]
    else:
        # Show file browser for date list
        w_date_container.children = [file_selector.widget]

w_mode.observe(on_mode_change, names='value')

# Progress and action
w_progress = widgets.IntProgress(min=0, max=1, value=0, description='Progress:', 
                                 bar_style='info', layout=widgets.Layout(width='60%'))

w_btn = widgets.Button(description='🚀 DOWNLOAD FSLE DATA', button_style='success',
                       layout=widgets.Layout(width='100%', height='50px'), icon='download')

w_output_log = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', padding='10px', max_height='400px', overflow='auto'))

def on_click_download(b):
    with w_output_log:
        clear_output()
        print("🚀 Starting FSLE download process...\n")
        
        lat_r = w_lat.value
        lon_r = w_lon.value
        mode = w_mode.value
        out_dir = folder_selector.get_selected_path()
        save_mode = w_save_mode.value
        
        print(f"📍 Region: Lat [{lat_r[0]}°, {lat_r[1]}°], Lon [{lon_r[0]}°, {lon_r[1]}°]")
        print(f"📂 Output: {out_dir}\n")
        
        if mode == 'Single Date':
            date_inp = w_date_single.value
        elif mode == 'Date Range':
            date_inp = (w_date_start.value, w_date_end.value)
        else:
            # Get file from file selector
            date_inp = file_selector.get_selected_file()
            if not date_inp:
                print("❌ No date list file selected! Please select a file first.")
                return
            print(f"📄 Date file: {date_inp}")
        
        success, msg = download_fsle_data(
            lat_range=lat_r,
            lon_range=lon_r,
            date_mode=mode,
            date_input=date_inp,
            output_dir=out_dir,
            save_mode=save_mode,
            progress_widget=w_progress
        )
        
        print("\n" + "="*50)
        print(msg)

w_btn.on_click(on_click_download)

# Assemble and display UI
ui = widgets.VBox([
    widgets.HTML("<h2>🌊 ODATIS/AVISO FSLE Downloader</h2>"),
    folder_selector.widget,
    widgets.HTML("<hr><b>Geographic Region</b>"),
    w_lat, w_lon,
    widgets.HTML("<hr><b>Date Selection</b>"),
    w_mode, w_date_container,
    widgets.HTML("<hr><b>Save Options</b>"),
    w_save_mode,
    widgets.HTML("<br>"),
    w_btn, w_progress,
    widgets.HTML("<hr><b>📋 Log:</b>"),
    w_output_log
])

display(ui)

In [ ]:
# =============================================================================
# CELL 4 (Optional): Merge individual files into single NetCDF
# =============================================================================

# Run this cell after downloading to combine all individual files

# fsle_folder = Path('/home/desenvolvedor/projetos/hackweek/2026-proj-Trawling4PACE/data/fsle')
# all_files = sorted(fsle_folder.glob('fsle_*.nc'))
# print(f"Found {len(all_files)} files")

# ds = xr.open_mfdataset(all_files, combine='by_coords')
# print(ds)

# ds.to_netcdf(fsle_folder / 'fsle_combined_all.nc')
# print("\n✅ Combined file saved!")

In [ ]:
# =============================================================================
# CELL 5 (Optional): Preview downloaded data
# =============================================================================

# ds = xr.open_dataset('fsle_data/fsle_20240307.nc')
# print(ds)
# ds['fsle_max'].plot(figsize=(10, 6))

In [8]:
# =============================================================================
# CELL 3: Complete download interface (all-in-one)
# =============================================================================

import os
import requests
import pandas as pd
import xarray as xr
import ipywidgets as widgets
from pathlib import Path
from datetime import datetime
from requests.auth import HTTPBasicAuth
from IPython.display import display, clear_output

# -----------------------------------------------------------------------------
# DOWNLOAD ENGINE
# -----------------------------------------------------------------------------
def parse_date_file(filepath):
    """
    Parse a text file containing dates, one per line.
    """
    dates = []
    skip_prefixes = ('#', '=', '-')
    skip_words = ('lista', 'total', 'date', 'datas', 'unique', 'unica')
    
    with open(filepath, 'r') as f:
        for line in f:
            # Remove inline comments (everything after #)
            if '#' in line:
                line = line.split('#')[0]
            
            line = line.strip()
            
            # Skip empty lines
            if not line:
                continue
            
            # Skip lines starting with special characters
            if line.startswith(skip_prefixes):
                continue
            
            # Skip header lines (case-insensitive)
            if any(line.lower().startswith(word) for word in skip_words):
                continue
            
            # Try to parse as date
            try:
                dates.append(pd.to_datetime(line))
            except:
                pass  # Skip unparseable lines silently
    
    return dates


def download_fsle_data(lat_range, lon_range, date_mode, date_input, output_dir, 
                       save_mode='individual', progress_widget=None):
    """
    Download FSLE data using direct NCSS URL construction with HTTP Basic Auth.
    Checks if file exists before downloading.
    """
    
    # 1. Parse and validate dates
    dates_to_download = []
    try:
        if date_mode == 'Single Date':
            dates_to_download = [pd.to_datetime(date_input)]
        elif date_mode == 'Date Range':
            start_date, end_date = date_input
            dates_to_download = pd.date_range(start=start_date, end=end_date, freq='D').tolist()
        elif date_mode == 'Date List File':
            if not os.path.exists(date_input):
                return False, f"❌ File not found: {date_input}"
            dates_to_download = parse_date_file(date_input)
    except Exception as e:
        return False, f"❌ Date processing error: {str(e)}"

    if not dates_to_download:
        return False, "❌ No valid dates found to download."
    
    # Sort dates chronologically
    dates_to_download = sorted(dates_to_download)
    print(f"📅 Found {len(dates_to_download)} valid dates in selection")

    # 2. Ensure output directory exists
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    print(f"📂 Output directory: {output_path.absolute()}")
    
    # 3. Check for existing files (LOGIC ADDED HERE)
    dates_to_process = []
    skipped_count = 0
    
    print("-" * 60)
    print("🔍 Checking existing files...")
    
    for dt in dates_to_download:
        # Define expected filename pattern
        expected_file = output_path / f"fsle_{dt.strftime('%Y%m%d')}.nc"
        
        if expected_file.exists():
            # WARN USER AND SKIP
            print(f"🚫 Skipping: {expected_file.name} (File already exists)")
            print(f"   (ℹ️ To download this date again, please delete the file from the folder)")
            skipped_count += 1
        else:
            dates_to_process.append(dt)
            
    print("-" * 60)
    
    if skipped_count > 0:
        print(f"⏭️ Total files skipped: {skipped_count}")
    
    if not dates_to_process:
        return True, f"✅ All {len(dates_to_download)} files already exist. No new downloads needed."
    
    print(f"\n🔄 Starting download for {len(dates_to_process)} NEW file(s)...")
    
    # 4. Setup authenticated session
    # ENSURE AVISO_USER AND AVISO_PASS ARE DEFINED IN PREVIOUS CELLS
    if 'AVISO_USER' not in globals() or 'AVISO_PASS' not in globals():
        # Fallback to hardcoded if not found (or raise error)
        # Using the credentials identified in previous steps
        user_auth = 'tds@odatis-ocean.fr'
        pass_auth = 'odatis'
    else:
        user_auth = AVISO_USER
        pass_auth = AVISO_PASS

    session = requests.Session()
    session.auth = HTTPBasicAuth(user_auth, pass_auth)
    
    if progress_widget:
        progress_widget.max = len(dates_to_process)
        progress_widget.value = 0
    
    downloaded_files = []
    failed_dates = []
    
    # Base URL for NCSS
    BASE_NCSS_URL = "https://tds-odatis.aviso.altimetry.fr/thredds/ncss/grid/dataset-duacs-dt-global-allsat-madt-fsle"

    # 5. Download loop
    for i, dt in enumerate(dates_to_process):
        try:
            t_start = dt.strftime('%Y-%m-%dT00:00:00Z')
            t_end = dt.strftime('%Y-%m-%dT23:59:59Z')
            
            params = {
                'var': ['fsle_max', 'theta_max'],
                'north': lat_range[1],
                'south': lat_range[0],
                'west': lon_range[0],
                'east': lon_range[1],
                'horizStride': 1,
                'time_start': t_start,
                'time_end': t_end,
                'accept': 'netcdf3',
                'addLatLon': 'true'
            }
            
            response = session.get(BASE_NCSS_URL, params=params, timeout=120)
            
            if response.status_code == 200:
                # Save directly to final location
                out_file = output_path / f"fsle_{dt.strftime('%Y%m%d')}.nc"
                with open(out_file, 'wb') as f:
                    f.write(response.content)
                
                downloaded_files.append(out_file)
                print(f"  ✓ Downloaded: {out_file.name}")
            else:
                print(f"  ⚠️ Failed {dt.strftime('%Y-%m-%d')}: HTTP {response.status_code} - {response.reason}")
                failed_dates.append(dt)
                if response.text:
                    print(f"     Server message: {response.text[:150]}...")
                    
        except Exception as e:
            print(f"  ❌ Error {dt.strftime('%Y-%m-%d')}: {str(e)}")
            failed_dates.append(dt)
        
        if progress_widget:
            progress_widget.value = i + 1

    # 6. Build result message
    total_files = len(downloaded_files) + skipped_count
    
    if not downloaded_files and skipped_count == 0:
        return False, "❌ No data was successfully downloaded."
    
    msg = f"\n✅ Process complete!\n"
    msg += f"📁 Output folder: {output_path}\n"
    msg += f"📥 Downloaded: {len(downloaded_files)}\n"
    msg += f"⏭️ Skipped (Existing): {skipped_count}\n"
    
    if failed_dates:
        msg += f"\n⚠️ Failed dates ({len(failed_dates)}): {[d.strftime('%Y-%m-%d') for d in failed_dates[:5]]}"
        if len(failed_dates) > 5:
            msg += f" ... and {len(failed_dates)-5} more"
    
    # 7. Optionally concatenate all files
    if save_mode == 'concatenated' and total_files > 1:
        print("\n💾 Concatenating all files into single NetCDF...")
        try:
            # We look for ALL files in the folder matching pattern, not just downloaded ones
            all_files = sorted(output_path.glob('fsle_*.nc'))
            if all_files:
                ds = xr.open_mfdataset(all_files, combine='by_coords')
                
                d_start = dates_to_download[0].strftime('%Y%m%d')
                d_end = dates_to_download[-1].strftime('%Y%m%d')
                concat_file = output_path / f"fsle_combined_{d_start}_to_{d_end}.nc"
                
                ds.to_netcdf(concat_file)
                msg += f"\n📦 Combined file created: {concat_file.name}"
        except Exception as e:
            msg += f"\n⚠️ Could not concatenate: {str(e)}"
    
    return True, msg


# -----------------------------------------------------------------------------
# FOLDER BROWSER CLASS
# -----------------------------------------------------------------------------
class FolderSelector:
    """Interactive folder selector with navigation and creation capabilities."""
    
    def __init__(self, start_path='.', title='📁 Output Folder'):
        self.current_path = Path(start_path).resolve()
        self.selected_path = self.current_path
        self.title = title
        
        self.path_display = widgets.HTML(value=self._format_path_html())
        
        self.folder_dropdown = widgets.Select(
            options=self._get_folder_options(),
            description='',
            layout=widgets.Layout(width='100%', height='120px')
        )
        
        self.up_btn = widgets.Button(description='⬆️ Up', button_style='info', layout=widgets.Layout(width='80px'))
        self.enter_btn = widgets.Button(description='📂 Enter', button_style='primary', layout=widgets.Layout(width='90px'))
        self.select_btn = widgets.Button(description='✅ Select', button_style='success', layout=widgets.Layout(width='90px'))
        
        self.new_folder_name = widgets.Text(placeholder='New folder name...', layout=widgets.Layout(width='180px'))
        self.create_btn = widgets.Button(description='➕ Create', button_style='warning', layout=widgets.Layout(width='90px'))
        
        self.selected_display = widgets.HTML(value=f"<b>Selected:</b> <code>{self.selected_path}</code>")
        
        self.up_btn.on_click(self._go_up)
        self.enter_btn.on_click(self._enter_folder)
        self.select_btn.on_click(self._select_current)
        self.create_btn.on_click(self._create_folder)
        
        self.widget = widgets.VBox([
            widgets.HTML(f"<h4>{self.title}</h4>"),
            self.path_display,
            self.folder_dropdown,
            widgets.HBox([self.up_btn, self.enter_btn, self.select_btn]),
            widgets.HBox([self.new_folder_name, self.create_btn]),
            self.selected_display
        ])
    
    def _format_path_html(self):
        return f"<b>Current:</b> <code>{self.current_path}</code>"
    
    def _get_folder_options(self):
        try:
            folders = ['[ . ] (current folder)']
            for item in sorted(self.current_path.iterdir()):
                if item.is_dir() and not item.name.startswith('.'):
                    folders.append(f"📁 {item.name}")
            return folders
        except PermissionError:
            return ['[ . ] (current folder)']
    
    def _refresh(self):
        self.path_display.value = self._format_path_html()
        self.folder_dropdown.options = self._get_folder_options()
        self.folder_dropdown.value = self.folder_dropdown.options[0]
    
    def _go_up(self, b):
        parent = self.current_path.parent
        if parent != self.current_path:
            self.current_path = parent
            self._refresh()
    
    def _enter_folder(self, b):
        selection = self.folder_dropdown.value
        if selection and not selection.startswith('[ . ]'):
            folder_name = selection.replace('📁 ', '')
            new_path = self.current_path / folder_name
            if new_path.is_dir():
                self.current_path = new_path
                self._refresh()
    
    def _select_current(self, b):
        self.selected_path = self.current_path
        self.selected_display.value = f"<b>✅ Selected:</b> <code>{self.selected_path}</code>"
    
    def _create_folder(self, b):
        name = self.new_folder_name.value.strip()
        if name:
            new_path = self.current_path / name
            try:
                new_path.mkdir(parents=True, exist_ok=True)
                self.new_folder_name.value = ''
                self.current_path = new_path
                self.selected_path = new_path
                self._refresh()
                self.selected_display.value = f"<b>✅ Created & Selected:</b> <code>{self.selected_path}</code>"
            except Exception as e:
                self.selected_display.value = f"<b>❌ Error:</b> {str(e)}"
    
    def get_selected_path(self):
        return str(self.selected_path)


# -----------------------------------------------------------------------------
# FILE BROWSER CLASS
# -----------------------------------------------------------------------------
class FileSelector:
    """Interactive file selector with navigation capabilities."""
    
    def __init__(self, start_path='.', title='📄 Select File', file_filter=None):
        self.current_path = Path(start_path).resolve()
        self.selected_file = None
        self.title = title
        self.file_filter = file_filter
        
        self.path_display = widgets.HTML(value=self._format_path_html())
        
        self.file_dropdown = widgets.Select(
            options=self._get_items(),
            description='',
            layout=widgets.Layout(width='100%', height='150px')
        )
        
        self.up_btn = widgets.Button(description='⬆️ Up', button_style='info', layout=widgets.Layout(width='80px'))
        self.enter_btn = widgets.Button(description='📂 Enter', button_style='primary', layout=widgets.Layout(width='90px'))
        self.select_btn = widgets.Button(description='✅ Select File', button_style='success', layout=widgets.Layout(width='120px'))
        
        self.selected_display = widgets.HTML(value="<b>Selected:</b> <i>No file selected</i>")
        
        self.up_btn.on_click(self._go_up)
        self.enter_btn.on_click(self._enter_folder)
        self.select_btn.on_click(self._select_file)
        
        self.widget = widgets.VBox([
            widgets.HTML(f"<h4>{self.title}</h4>"),
            self.path_display,
            self.file_dropdown,
            widgets.HBox([self.up_btn, self.enter_btn, self.select_btn]),
            self.selected_display
        ])
    
    def _format_path_html(self):
        return f"<b>Current:</b> <code>{self.current_path}</code>"
    
    def _get_items(self):
        """Get folders and files in current directory."""
        try:
            items = []
            folders = []
            files = []
            
            for item in sorted(self.current_path.iterdir()):
                if item.name.startswith('.'):
                    continue
                if item.is_dir():
                    folders.append(f"📁 {item.name}")
                elif item.is_file():
                    # Apply file filter if specified
                    if self.file_filter:
                        if item.suffix.lower() in self.file_filter:
                            files.append(f"📄 {item.name}")
                    else:
                        files.append(f"📄 {item.name}")
            
            return folders + files if (folders or files) else ['(empty folder)']
        except PermissionError:
            return ['(permission denied)']
    
    def _refresh(self):
        self.path_display.value = self._format_path_html()
        self.file_dropdown.options = self._get_items()
        if self.file_dropdown.options:
            self.file_dropdown.value = self.file_dropdown.options[0]
    
    def _go_up(self, b):
        parent = self.current_path.parent
        if parent != self.current_path:
            self.current_path = parent
            self._refresh()
    
    def _enter_folder(self, b):
        selection = self.file_dropdown.value
        if selection and selection.startswith('📁'):
            folder_name = selection.replace('📁 ', '')
            new_path = self.current_path / folder_name
            if new_path.is_dir():
                self.current_path = new_path
                self._refresh()
    
    def _select_file(self, b):
        selection = self.file_dropdown.value
        if selection and selection.startswith('📄'):
            file_name = selection.replace('📄 ', '')
            self.selected_file = self.current_path / file_name
            self.selected_display.value = f"<b>✅ Selected:</b> <code>{self.selected_file}</code>"
        else:
            self.selected_display.value = "<b>⚠️</b> Please select a file (📄), not a folder"
    
    def get_selected_file(self):
        """Return the selected file path as string, or None if no file selected."""
        return str(self.selected_file) if self.selected_file else None


# -----------------------------------------------------------------------------
# USER INTERFACE
# -----------------------------------------------------------------------------
style = {'description_width': 'initial'}

# Create folder selector for output
folder_selector = FolderSelector(start_path='.', title='📁 Output Folder')

# Create file selector for date list
file_selector = FileSelector(start_path='.', title='📄 Date List File', file_filter=['.txt', '.csv', '.dat'])

# Geographic selection
w_lat = widgets.FloatRangeSlider(
    value=[33, 45], min=-90, max=90, step=0.5,
    description='Latitude (S → N):', style=style,
    layout=widgets.Layout(width='60%'), continuous_update=False
)

w_lon = widgets.FloatRangeSlider(
    value=[-77, -64], min=-180, max=180, step=0.5,
    description='Longitude (W → E):', style=style,
    layout=widgets.Layout(width='60%'), continuous_update=False
)

# Date selection
w_mode = widgets.Dropdown(
    options=['Single Date', 'Date Range', 'Date List File'],
    value='Single Date', description='Date Mode:', style=style
)

w_date_single = widgets.DatePicker(description='Select Date:', value=datetime(2024, 3, 30), style=style)
w_date_start = widgets.DatePicker(description='Start Date:', value=datetime(2024, 3, 30), style=style)
w_date_end = widgets.DatePicker(description='End Date:', value=datetime(2024, 4, 1), style=style)
w_date_range_box = widgets.HBox([w_date_start, w_date_end])

# Save mode selector
w_save_mode = widgets.RadioButtons(
    options=[
        ('Individual files (one per day, resumable)', 'individual'),
        ('Single concatenated file', 'concatenated')
    ],
    value='individual',
    description='Save mode:',
    style=style
)

# Container for date inputs (will switch between picker and file browser)
w_date_container = widgets.VBox([w_date_single])

def on_mode_change(change):
    if change['new'] == 'Single Date':
        w_date_container.children = [w_date_single]
    elif change['new'] == 'Date Range':
        w_date_container.children = [w_date_range_box]
    else:
        # Show file browser for date list
        w_date_container.children = [file_selector.widget]

w_mode.observe(on_mode_change, names='value')

# Progress and action
w_progress = widgets.IntProgress(min=0, max=1, value=0, description='Progress:', 
                                 bar_style='info', layout=widgets.Layout(width='60%'))

w_btn = widgets.Button(description='🚀 DOWNLOAD FSLE DATA', button_style='success',
                       layout=widgets.Layout(width='100%', height='50px'), icon='download')

w_output_log = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', padding='10px', max_height='400px', overflow='auto'))

def on_click_download(b):
    with w_output_log:
        clear_output()
        print("🚀 Starting FSLE download process...\n")
        
        lat_r = w_lat.value
        lon_r = w_lon.value
        mode = w_mode.value
        out_dir = folder_selector.get_selected_path()
        save_mode = w_save_mode.value
        
        print(f"📍 Region: Lat [{lat_r[0]}°, {lat_r[1]}°], Lon [{lon_r[0]}°, {lon_r[1]}°]")
        print(f"📂 Output: {out_dir}\n")
        
        if mode == 'Single Date':
            date_inp = w_date_single.value
        elif mode == 'Date Range':
            date_inp = (w_date_start.value, w_date_end.value)
        else:
            # Get file from file selector
            date_inp = file_selector.get_selected_file()
            if not date_inp:
                print("❌ No date list file selected! Please select a file first.")
                return
            print(f"📄 Date file: {date_inp}")
        
        success, msg = download_fsle_data(
            lat_range=lat_r,
            lon_range=lon_r,
            date_mode=mode,
            date_input=date_inp,
            output_dir=out_dir,
            save_mode=save_mode,
            progress_widget=w_progress
        )
        
        print("\n" + "="*50)
        print(msg)

w_btn.on_click(on_click_download)

# Assemble and display UI
ui = widgets.VBox([
    widgets.HTML("<h2>🌊 ODATIS/AVISO FSLE Downloader</h2>"),
    folder_selector.widget,
    widgets.HTML("<hr><b>Geographic Region</b>"),
    w_lat, w_lon,
    widgets.HTML("<hr><b>Date Selection</b>"),
    w_mode, w_date_container,
    widgets.HTML("<hr><b>Save Options</b>"),
    w_save_mode,
    widgets.HTML("<br>"),
    w_btn, w_progress,
    widgets.HTML("<hr><b>📋 Log:</b>"),
    w_output_log
])

display(ui)